In [67]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import os
import numpy as np
import pandas as pd
import os, random
from glob import glob
import cv2
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [68]:
import torch
import torch.nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [69]:
# 데이터셋 루트 폴더
dataset_root = 'C:/Users/User/data/result/'

# 모든 클래스 폴더 가져오기
class_folders = [f for f in os.listdir(dataset_root) if os.path.isdir(os.path.join(dataset_root, f))]

# 각 클래스 폴더 내의 이미지 수 계산
total_images = 0
for class_folder in class_folders:
    class_path = os.path.join(dataset_root, class_folder)
    images_in_class = [f for f in os.listdir(class_path) if f.endswith('.jpg') or f.endswith('.png')]
    total_images += len(images_in_class)

# 결과 출력
print(f'클래스 폴더 수: {len(class_folders)}')
print(f'총 이미지 수: {total_images}')

클래스 폴더 수: 3
총 이미지 수: 6133


In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder

# 이미지의 사이즈를 변환 후 텐서로 변환 
transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

#폴더의 이미지를 로딩, 라벨링
train_dataset = ImageFolder(root='C:/Users/User/data/result/', transform=transforms)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [71]:
class_names = train_dataset.classes
# 학습 결과 시각화
train_losses = []  # 학습 손실값을 기록할 리스트
train_accuracies = []  # 학습 정확도를 기록할 리스트
save_interval = 1

# 에포크마다 모델 저장을 위한 경로와 파일명 지정
model_save_path = 'C:/Users/User/Desktop/model_result/models'  # 저장할 디렉토리 경로
model_name = 'model'  # 저장할 모델 파일명 (에포크 번호가 자동으로 붙음)

# 저장할 폴더가 없다면 생성
os.makedirs(model_save_path, exist_ok=True)

In [72]:
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    return 100 * correct / total

In [73]:
class CNNModel(nn.Module) :
    # 모델의 레이어
    def __init__(self, num_classes) :
        super(CNNModel, self).__init__()
        #두개의 합성곱, 풀링 레이어(pool), 완전연결레이어(fc1)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(16 * 16 * 64, num_classes) #이미지 사이즈가 64x64로 변경됨

    # 입력데이터의 연산 수행 후 출력
    def forward(self, x) :
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 16 * 64)
        x = self.fc1(x)

        return x

model = CNNModel(num_classes=3)
model.to(device)
#다중클래스 분류에서 사용하는 손실함수
criterion = nn.CrossEntropyLoss()
# 알고리즘(adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

#학습 수행(epoch = 30)
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for data in train_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    if (epoch + 1) % save_interval == 0:
        model_filename = f'{model_name}_epoch_{epoch+1}.pt'
        model_filepath = os.path.join(model_save_path, model_filename)
        torch.save(model.state_dict(), model_filepath)


Epoch [1/30] - Loss: 0.8670, Accuracy: 58.23%
Epoch [2/30] - Loss: 0.6903, Accuracy: 67.78%
Epoch [3/30] - Loss: 0.5847, Accuracy: 74.21%
Epoch [4/30] - Loss: 0.5170, Accuracy: 77.68%
Epoch [5/30] - Loss: 0.4601, Accuracy: 80.91%
Epoch [6/30] - Loss: 0.4223, Accuracy: 83.04%
Epoch [7/30] - Loss: 0.3664, Accuracy: 85.60%
Epoch [8/30] - Loss: 0.3446, Accuracy: 86.53%
Epoch [9/30] - Loss: 0.3179, Accuracy: 87.53%
Epoch [10/30] - Loss: 0.3067, Accuracy: 88.50%
Epoch [11/30] - Loss: 0.2855, Accuracy: 89.66%
Epoch [12/30] - Loss: 0.2583, Accuracy: 90.54%
Epoch [13/30] - Loss: 0.2533, Accuracy: 90.62%
Epoch [14/30] - Loss: 0.2390, Accuracy: 91.37%
Epoch [15/30] - Loss: 0.2168, Accuracy: 92.12%
Epoch [16/30] - Loss: 0.2128, Accuracy: 92.48%
Epoch [17/30] - Loss: 0.2134, Accuracy: 92.29%
Epoch [18/30] - Loss: 0.1864, Accuracy: 93.64%
Epoch [19/30] - Loss: 0.1913, Accuracy: 93.10%
Epoch [20/30] - Loss: 0.1781, Accuracy: 94.16%
Epoch [21/30] - Loss: 0.1612, Accuracy: 94.34%
Epoch [22/30] - Loss: 

In [74]:

def plt_show_loss(losses) :
    plt.plot(losses.history['loss'])
    plt.plot(losses.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc=0)

def plt_show_acc(accuracies) :
    plt.plot(accuracies.history['accuracy'])
    plt.plot(accuracies.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc=0)

plt_show_loss(train_losses)
plt.show()

plt_show_acc(train_accuracies)
plt.show()

AttributeError: 'list' object has no attribute 'history'

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image

# 폴더 내의 모든 이미지 파일 경로 가져오기
folder_path = "C:/Users/User/Desktop/bigdata/imgs/test/"  # 이미지 폴더 경로
image_files = glob.glob(os.path.join(folder_path, "*.jpg"))  # jpg 확장자를 가진 이미지 파일들의 경로 리스트
class_names = ["Mink", "Porpoise"]

# 모델을 eval 상태로 설정
model.eval()

# 변환 함수 정의
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

for img_path in image_files:
    # 이미지 불러오기
    img = Image.open(img_path)
    img = preprocess(img)
    img = torch.unsqueeze(img, 0)

    # 모델로 예측하기
    with torch.no_grad():
        predictions = model(img)
        predictions = F.softmax(predictions[0], dim=0) * 100
        class_probs = [prob.item() for prob in predictions]

    # 예측 결과 시각화하기
    for i, class_name in enumerate(class_names):
        print(f"Class: {class_name}, Probability: {class_probs[i]:.2f}%")

    predicted_class = torch.argmax(predictions).item()
    print(f"Predicted Class: {class_names[predicted_class]}")
    print("="*30)
